# This doc is for customizing pycairo output, with input drawn from the tsv file output of the main function. Make sure the correct output was selected for tsv file population!

In [280]:
import cairo
import re

In [275]:
with open("Motifoutput.tsv", "r") as fh:
    view = fh.readlines()
print(view[0:5])

['INSR chr19:7150261-7150808 (reverse complement)\n', 'Sequence length: 548\n', 'Exon positions\t[(277, 312)]\n', '\n', 'Motif    Occurances  Forward positions   Reverse positions\n']


# Change image parameters here. For custom colors, scroll to next header.

In [276]:
width, height = 800, 500
margin = 20
fontsize = 13

In [281]:
def draw(dictionary, width, height, margin):
    length = dictionary.pop("length")                                             #Assigns the length from features dictionary input as a variable. NOTE: Pop is used throughout to reduce dictionary to motifs only by the end of the function call. (Easy for drawing based on each motif in dictionary)
    adjustedwidth = (width - 2*margin)/int(length)                              #For scaling the image based on input sequence length; this varible is multiplied by index coordinates
    header = dictionary.pop("header")                                             #Assigns header variable for printing in image
    title = header                                                              #Seperate varible from header; manipulated if header cannot be used as a file name

    #Makes sure the svg name is able to be saved.
    forbid = ["\\", "/", ":", "*", "?", "\"", "<", ">", "|"]                    #All forbidden characters for file naming
    for item in forbid:
        if item in title:
            title = re.sub(item, '.', title, flags=re.IGNORECASE)               #Replaces all forbidden characters with "."; ensures no overwite occurs from shortening since headers should be unique
    
    surface = cairo.SVGSurface(title+".motifoutput.svg", width, height)
    context = cairo.Context(surface)  ## generate surface
    
    context.set_source_rgb(1.0, 1.0, 1.0)
    context.rectangle(0, height/2, width, height/2) #x initial, y inital (upper), width of box, height of box
    context.fill()
    context.set_source_rgb(0.0, 0.0, 0.0)
    context.move_to(margin, fontsize+margin)
    context.set_font_size(fontsize)
    context.show_text("Sequence: "+header+", length: "+length+" bases")
    context.move_to(margin, 3*fontsize+margin)
    context.show_text("Motifs:")
    context.move_to(margin, height-2*fontsize-margin)
    context.show_text("Exons depicted as black boxes. Introns depicted as thin black lines. Motifs are colored lines above and below the sequence.")
    context.move_to(margin, height-1*fontsize-margin)
    context.show_text("Motifs above the sequence are aligned to the input sequence. Motifs below align to the reverse compliment of the input sequence.")
    
    context.move_to(margin, height/2) #initial x, y coordinates
    context.line_to((margin+int(length)*adjustedwidth), height/2) # Line destination equal to full sequence length
    context.stroke() #draws lines
    
    exons = dictionary.pop("exon")
    exons = exons.strip("[")
    exons = exons.strip("]")
    exons = exons.split("), (")
    
    
    for exon in exons:
        exon = exon.strip(")")
        exon = exon.strip("(")
        exon = exon.split(", ")
        context.rectangle((margin+int(exon[0])*adjustedwidth),240,(int(exon[1])-int(exon[0]))*adjustedwidth,20) #x initial, y inital (upper), width of box, height of box
        context.fill() ## fill rectangle
        
    colorminus = 2.5/(len(dictionary)+1)
    loop = 0
    colorlist = [0.90, 0.90, 0.90]
    
    for motif in dictionary:
        colorlist[loop%3] -= colorminus
        loop += 1
        context.set_source_rgb(colorlist[0], colorlist[1], colorlist[2])
        
        context.move_to(margin, (loop+3)*fontsize+margin)
        context.show_text(motif)

        motif = dictionary[motif]
        fwdmotif = motif[0]
        fwdmotif = fwdmotif.strip("[(")
        fwdmotif = fwdmotif.strip(")]")
        fwdmotif = fwdmotif.split("), (")
        revmotif = motif[1]
        revmotif = revmotif.strip("[(")
        revmotif = revmotif.strip(")]")
        revmotif = revmotif.split("), (")
        
        
        
        if fwdmotif != ['']:
            for span in fwdmotif:
                span = span.split(", ")             
                context.move_to(margin+(int(span[0])*adjustedwidth), height/2-20-(10*loop)) #initial x, y coordinates (from left)
                context.line_to(margin+(int(span[1])*adjustedwidth), height/2-20-(10*loop)) # Line destination equal to full sequence length
                context.stroke() #draws lines
                
        if revmotif != ['']:
            for span in revmotif:
                span = span.split(", ")
                context.move_to(width-(margin+(int(span[0])*adjustedwidth)), height/2+20+(10*loop)) #initial x, y coordinates (from right, revcomp coordinates)
                context.line_to(width-(margin+(int(span[1])*adjustedwidth)), height/2+20+(10*loop)) # Line destination equal to full sequence length
                context.stroke() #draws lines
      
    surface.finish() # end, closes image
                

dictionary = {}
dictionary["header"] = "first"

for line in view:
    if line != "\n" and line != "Motif    Occurances  Forward positions   Reverse positions\n":
        if "\t" not in line and dictionary["header"] == "first":
            dictionary["header"] = line.strip("\n")
        elif "length" in line:
            line=line.strip("\n")
            line=line.split(" ")
            dictionary["length"] = line[2]
        elif "\t" not in line:
            draw(dictionary, width, height, margin)
            dictionary = {}
            dictionary["header"] = line.strip("\n")
        elif "Exon" in line:
            line = line.strip("\n")
            line = line.split("\t")
            dictionary["exon"] = line[1]
        else:
            line = line.strip("\n")
            line = line.split("\t")
            dictionary[line[0]]= (line[2], line[3])
draw(dictionary, width, height, margin)

# For custom colors, input rbg codes below. First color will be the sequence/exon/text heading color, and subsequent colors will be for each following motif.

In [272]:
colorwheel = [
    (0.0, 0.0, 0.0),
    (0.9, 0.5, 0.5),
    (0.2, 0.8, 0.2),
    (1.0, 0.8, 0.5),
    (0.9, 0.5, 0.5),
    (0.2, 0.8, 0.2),
    (1.0, 0.8, 0.5),
    (0.9, 0.5, 0.5),
    (0.2, 0.8, 0.2)
]

#Add as many colors as motifs+1!

In [273]:
def draw(dictionary, width, height, margin):
    
    length = dictionary.pop("length")                                             #Assigns the length from features dictionary input as a variable. NOTE: Pop is used throughout to reduce dictionary to motifs only by the end of the function call. (Easy for drawing based on each motif in dictionary)
    adjustedwidth = (width - 2*margin)/int(length)                              #For scaling the image based on input sequence length; this varible is multiplied by index coordinates
    header = dictionary.pop("header")                                             #Assigns header variable for printing in image
    title = header                                                              #Seperate varible from header; manipulated if header cannot be used as a file name

    #Makes sure the svg name is able to be saved.
    forbid = ["\\", "/", ":", "*", "?", "\"", "<", ">", "|"]                    #All forbidden characters for file naming
    for item in forbid:
        if item in title:
            title = re.sub(item, '.', title, flags=re.IGNORECASE)               #Replaces all forbidden characters with "."; ensures no overwite occurs from shortening since headers should be unique
    surface = cairo.SVGSurface(title+".motifoutput.svg", width, height)
    context = cairo.Context(surface)  ## generate surface
    
    context.set_source_rgb(1.0, 1.0, 1.0)
    context.rectangle(0, height/2, width, height/2) #x initial, y inital (upper), width of box, height of box
    context.fill()
    context.set_source_rgb(colorwheel[0][0], colorwheel[0][1], colorwheel[0][2])
    context.move_to(margin, fontsize+margin)
    context.set_font_size(fontsize)
    context.show_text("Sequence: "+header+", length: "+length+" bases")
    context.move_to(margin, 3*fontsize+margin)
    context.show_text("Motifs:")
    context.move_to(margin, height-2*fontsize-margin)
    context.show_text("Exons depicted as black boxes. Introns depicted as thin black lines. Motifs are colored lines above and below the sequence.")
    context.move_to(margin, height-1*fontsize-margin)
    context.show_text("Motifs above the sequence are aligned to the input sequence. Motifs below align to the reverse compliment of the input sequence.")
    
    context.move_to(margin, height/2) #initial x, y coordinates
    context.line_to((margin+int(length)*adjustedwidth), height/2) # Line destination equal to full sequence length
    context.stroke() #draws lines
    
    exons = dictionary.pop("exon")
    exons = exons.strip("[")
    exons = exons.strip("]")
    exons = exons.split("), (")
    
    
    for exon in exons:
        exon = exon.strip(")")
        exon = exon.strip("(")
        exon = exon.split(", ")
        context.rectangle((margin+int(exon[0])*adjustedwidth),240,(int(exon[1])-int(exon[0]))*adjustedwidth,20) #x initial, y inital (upper), width of box, height of box
        context.fill() ## fill rectangle
        
    loop = 0
    
    for motif in dictionary:
        loop += 1
        context.set_source_rgb(colorwheel[loop][0], colorwheel[loop][1], colorwheel[loop][2])
        
        context.move_to(margin, (loop+3)*fontsize+margin)
        context.show_text(motif)

        motif = dictionary[motif]
        fwdmotif = motif[0]
        fwdmotif = fwdmotif.strip("[(")
        fwdmotif = fwdmotif.strip(")]")
        fwdmotif = fwdmotif.split("), (")
        revmotif = motif[1]
        revmotif = revmotif.strip("[(")
        revmotif = revmotif.strip(")]")
        revmotif = revmotif.split("), (")
        
        
        
        if fwdmotif != ['']:
            for span in fwdmotif:
                span = span.split(", ")             
                context.move_to(margin+(int(span[0])*adjustedwidth), height/2-20-(10*loop)) #initial x, y coordinates (from left)
                context.line_to(margin+(int(span[1])*adjustedwidth), height/2-20-(10*loop)) # Line destination equal to full sequence length
                context.stroke() #draws lines
                
        if revmotif != ['']:
            for span in revmotif:
                span = span.split(", ")
                context.move_to(width-(margin+(int(span[0])*adjustedwidth)), height/2+20+(10*loop)) #initial x, y coordinates (from right, revcomp coordinates)
                context.line_to(width-(margin+(int(span[1])*adjustedwidth)), height/2+20+(10*loop)) # Line destination equal to full sequence length
                context.stroke() #draws lines
      
    surface.finish() # end, closes image
                

dictionary = {}
dictionary["header"] = "first"

for line in view:
    if line != "\n" and line != "Motif    Occurances  Forward positions   Reverse positions\n":
        if "\t" not in line and dictionary["header"] == "first":
            dictionary["header"] = line.strip("\n")
        elif "length" in line:
            line=line.strip("\n")
            line=line.split(" ")
            dictionary["length"] = line[2]
        elif "\t" not in line:
            draw(dictionary, width, height, margin)
            dictionary = {}
            dictionary["header"] = line.strip("\n")
        elif "Exon" in line:
            line = line.strip("\n")
            line = line.split("\t")
            dictionary["exon"] = line[1]
        else:
            line = line.strip("\n")
            line = line.split("\t")
            dictionary[line[0]]= (line[2], line[3])
draw(dictionary, width, height, margin)